In [1]:
# Importing the required modules

import googlemaps
from datetime import datetime

import json
import requests
from urllib.request import urlopen
import csv
import time
from urllib.parse import urlencode

In [2]:
# Function that returns the JSON script listing up to 20 businesses
# and their information

def MapBus(city, business, pricemin,pricemax,nexter,keys):
  # Making the URL
    if (nexter != ''):
        TheURL = ('https://maps.googleapis.com/maps/api/place/textsearch/json'
           '?pagetoken=%s'
           '&key=%s') % (nexter, keys)
    else:
        TheURL = ('https://maps.googleapis.com/maps/api/place/textsearch/json'
                    '?query=%s'
                     '&type=%s'
                     '&minprice=%s'
                     '&maxprice=%s'
                     '&key=%s') % (city, business, pricemin, pricemax, keys)

  # Using the URL to return a JSON value
    answer = urlopen(TheURL)
    uneditedJSON = answer.read()
    FinalJSON = json.loads(uneditedJSON)
    return FinalJSON

In [3]:
# Fill in each of these values based on what businesses you are looking
# for, where, and the csv file you want to export to
# Note the key value, which is required and can be obtained by
# using the Google Maps API. The project with the key 
# should have the Google Maps Places API downloaded.

# Make sure to fill out the key and data_file variables below.
# Otherwise the code will not work!

# Enter a place like you would in Google Maps
place = input('What place do you want information for?')

# Enter a type like 'restaurant', 'hospital', etc.
# These correspond to Google Maps API's types
business = input('What kind of business do you want information for?')

# Enter a digit for the prices
minprice1 = int(input("What is the lowest price level to filter the business by? Select a digit between 0 and 4?"))
maxprice1 = int(input("What is the highest price level to filter the business by? Select a digit between 0 and 4?"))

# Enter your key here in the code (not a prompt)
key = 'Your_Key_Here'

# Enter your data file path here in the code
data_file = open('Your_CSV_Here.csv', 'w')

# Changes spaces to pluses for inputting into URL
place = place.replace(' ','+')
business = business.replace(' ','+')

# Finds original 20 businesses and their information
csvwriter = csv.writer(data_file)
full_data = MapBus(place,business,minprice1,maxprice1,'',key)
sub_result = full_data['results']
count = 0

# Creates header
csvwriter.writerow(['Name','Latitude','Longitude'])

# Goes through up to 3 pages (limited by Google API) of up to 20 businesses
# or the total number of businesses (whichever is less) and writes 
# the name, latitude, and longitude to the csv file
while(full_data['status'] == 'OK' and count < 3):
    try:
        for i in range(0,len(sub_result)):
            csvwriter.writerow([sub_result[i]['name'], sub_result[i]['geometry']['location']['lat'], 
                       sub_result[i]['geometry']['location']['lng']])
        time.sleep(5)
        count += 1
        full_data = MapBus(place,business,minprice1,maxprice1,full_data['next_page_token'],key)
        sub_result = full_data['results']
    except KeyError:
        break
    

# In case fewer than 3 pages are used, this is the final page 
# of data added to the csv file
if (full_data['status'] == 'OK' and count < 3):
        MapBus(place,business,minprice1,maxprice1,'',key)

    
data_file.close()

What place do you want information for?Edison
What kind of business do you want information for?restaurant
What is the lowest price level to filter the business by? Select a digit between 0 and 4?0
What is the highest price level to filter the business by? Select a digit between 0 and 4?4
